# apriori analysis

### after clustering,it will generate k classes,each class will have a list of user_ids inside that class,next step is to apply basket anaysis on each classification,to get the rules of each class

In [1]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')

# fresh fruits:24
# fresh vegetables:83
# packaged vegetables fruits:122

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def Apriori(user_product_part1):
    
    user_product_dic={}
    for item in user_product_part1.itertuples():
        if(user_product_dic.get(item[2])):
            user_product_dic[item[2]].append(item[3])
        else:
            user_product_dic[item[2]] = [] 
            user_product_dic[item[2]].append(item[3])
    transaction = []
    for item in user_product_dic:
        transaction.append(user_product_dic[item])
#     print('transaction',transaction)
    te = TransactionEncoder()
    te_ary = te.fit(transaction).transform(transaction)
#     print('te_ary',te_ary)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    return df

In [3]:
def transform_associationRule_to_product(association_rule):
    
    relationship_rule = association_rule[['antecedents','consequents']]
    arr_relation=relationship_rule.values
    final_rules = []
    for item in arr_relation:
        pre_product_Arr = []
        after_product_Arr = []
        pre = item[0]
        after = item[1]
        for pre_item in pre:
            pre_product_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
        for after_item in after:
            after_product_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
        tup = (pre_product_Arr,after_product_Arr)
        final_rules.append(tup)
    return final_rules

In [4]:
def transform_frequentItem_to_product(frequent_itemsets):
    arr_relation=frequent_itemsets.values
    final_arr = []
    for item in arr_relation:
        arr = []
        for it in item[1]:
            arr.append(products[products['product_id' ]== it]['product_name'].values[0])
        final_arr.append(arr)
    return final_arr

#  association items of all users (no any change)

In [5]:
order_user_id = orders[['order_id','user_id']]

order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
# order_products_id = order_products_id2
order_products_id
user_product_id=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')
user_product_id

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [6]:
df = Apriori(user_product_id)
frequent_itemsets_all = apriori(df, min_support=0.1, use_colnames=True)
association_rule_all = association_rules(frequent_itemsets_all, metric="confidence", min_threshold=0.1)

In [7]:
transform_associationRule_to_product(association_rule_all)

[(['Bag of Organic Bananas'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Banana']),
 (['Banana'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Organic Hass Avocado']),
 (['Organic Hass Avocado'], ['Bag of Organic Bananas']),
 (['Banana'], ['Strawberries']),
 (['Strawberries'], ['Banana']),
 (['Organic Strawberries'], ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Banana']),
 (['Banana'], ['Organic Strawberries']),
 (['Limes'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Limes']),
 (['Organic Strawberries'], ['Organic Blueberries']),
 (['Organic Blueberries'], ['Organic Strawberries']),
 (['Organic Hass Avocado'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Organic Hass Avocado']),
 (['Organic Stra

In [8]:
s = frequent_itemsets_all.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)


[['Banana'],
 ['Bag of Organic Bananas'],
 ['Organic Strawberries'],
 ['Organic Baby Spinach'],
 ['Large Lemon'],
 ['Limes'],
 ['Strawberries'],
 ['Organic Hass Avocado'],
 ['Organic Avocado'],
 ['Organic Blueberries'],
 ['Organic Garlic'],
 ['Organic Yellow Onion'],
 ['Organic Zucchini'],
 ['Organic Raspberries'],
 ['Bag of Organic Bananas', 'Organic Strawberries'],
 ['Yellow Onions'],
 ['Cucumber Kirby'],
 ['Organic Grape Tomatoes'],
 ['Organic Strawberries', 'Organic Baby Spinach'],
 ['Seedless Red Grapes']]

#  association items of all users (remove asiles : fresh fruits(24),fresh vegetables(83),packaged vegetables fruits(122))

### find the top 3 aisles [fresh fruits,fresh vegetables,packaged vegetables fruits]

In [9]:
aisle_ids = []
aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for item in aisle_arr:
    aisle_ids.append(item[0])

product_ids = []
product_arr = products[products['aisle_id'].isin(aisle_ids)].values
for item in product_arr:
    product_ids.append(item[0])
product_ids
aisle_ids

[24, 83, 123]

In [10]:
user_product_id = user_product_id[~user_product_id['product_id'].isin(product_ids)]

In [11]:
df = Apriori(user_product_id)
t1 = time.time()
frequent_itemsets = apriori(df, min_support=0.08, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

time t2 - t1 29.57947611808777


In [12]:
association_rule = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.08)
# association_rule

In [13]:
transform_associationRule_to_product(association_rule)

[]

In [14]:
s = frequent_itemsets.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Blueberries'],
 ['Organic Whole Milk'],
 ['Bunched Cilantro'],
 ['Original Hummus'],
 ['Raspberries'],
 ['Grated Parmesan'],
 ['Sparkling Water Grapefruit'],
 ['Boneless Skinless Chicken Breasts'],
 ['Organic Half & Half'],
 ['Half & Half'],
 ['100% Whole Wheat Bread']]

# after removing frequent asiles and apply association rule on each user classification 

In [15]:
user_classification = pd.read_csv('../customer_classificationV2.csv')
user_classification = user_classification[['user_id','label']]
user0 = user_classification[user_classification['label']==0]
user1 = user_classification[user_classification['label']==1]
user2 = user_classification[user_classification['label']==2]
user3 = user_classification[user_classification['label']==3]
user4 = user_classification[user_classification['label']==4]

In [73]:
def apply_apriori_fun(dataset,minSup,minCon):
    df = Apriori(dataset)
    t1 = time.time()
    frequent_itemsets0 = apriori(df, min_support=minSup, use_colnames=True)
    t2 = time.time()
    print('time t2 - t1',t2-t1)

    association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=minCon)
    # association_rule0
    
    s = frequent_itemsets0.sort_values('support',ascending=False)[:20]
    print('top frequent items')
    frequent_items = transform_frequentItem_to_product(s)
    for item in frequent_items:
        print(item)
    
#     return association_rule0[['antecedents','consequents']]
    return transform_associationRule_to_product(association_rule0)

#  association items of clustering 0

In [74]:
user_product_part0 = user_product_id[user_product_id['user_id'].isin(user0['user_id']) ]

### top rules and top items of train data (70%)

In [75]:
x0=apply_apriori_fun(user_product_part0[:int(len(user_product_part0)*0.7)],0.08,0.1)
x0

time t2 - t1 2.6199440956115723
top frequent items
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Half & Half']
['Organic Black Beans']
['Organic Basil']
['Boneless Skinless Chicken Breasts']
['Feta Cheese Crumbles']
['Organic Garbanzo Beans']
['Organic Half & Half']
['Organic Thyme']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Coconut Milk']
['Organic Sour Cream']


[(['Organic Cilantro'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Cilantro']),
 (['Organic Whole Milk'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Whole Milk']),
 (['Bunched Cilantro'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Bunched Cilantro']),
 (['Bunched Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Extra Virgin Olive Oil']),
 (['Grated Parmesan'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Grated Parmesan']),
 (['Organic Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Cilantro']),
 (['Grated Parmesan'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Grated Parmesan'

### top rules and top items of test data (30%)

In [76]:
apply_apriori_fun(user_product_part0[-int(len(user_product_part0)*0.3):],0.08,0.1)

time t2 - t1 0.820833683013916
top frequent items
['Extra Virgin Olive Oil']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Organic Black Beans']
['Half & Half']
['Organic Half & Half']
['Boneless Skinless Chicken Breasts']
['Organic Basil']
['Feta Cheese Crumbles']
['Organic Coconut Milk']
['Organic Garbanzo Beans']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Whole String Cheese']
['Raspberries']


[(['Organic Whole Milk'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Whole Milk']),
 (['Bunched Cilantro'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Bunched Cilantro']),
 (['Bunched Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Extra Virgin Olive Oil']),
 (['Organic Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Cilantro']),
 (['Grated Parmesan'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Grated Parmesan'])]

#  association items of clustering 1

In [77]:
user_product_part1 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]

### top rules and top items of train data (70%)

In [ ]:
x1=apply_apriori_fun(user_product_part1[:int(len(user_product_part1)*0.7)],0.13,0.1)
x1

### top rules and top items of test data (30%)

In [ ]:
apply_apriori_fun(user_product_part1[-int(len(user_product_part1)*0.3):],0.13,0.1)

#  association items of clustering 2

In [ ]:
user_product_part2 = user_product_id[user_product_id['user_id'].isin(user2['user_id']) ]

### top rules and top items of train data (70%)

In [ ]:
x2=apply_apriori_fun(user_product_part2[:int(len(user_product_part2)*0.7)],0.08,0.1)
x2

### top rules and top items of test data (30%)

In [ ]:
apply_apriori_fun(user_product_part2[-int(len(user_product_part2)*0.3):],0.08,0.1)

#  association items of clustering 3

In [ ]:
user_product_part3 = user_product_id[user_product_id['user_id'].isin(user3['user_id']) ]

### top rules and top items of train data (70%)

In [ ]:
x3=apply_apriori_fun(user_product_part3[:int(len(user_product_part3)*0.7)],0.2,0.2)
x3

### top rules and top items of test data (30%)

In [ ]:
apply_apriori_fun(user_product_part3[-int(len(user_product_part3)*0.3):],0.2,0.2)

#  association items of clustering 4

In [ ]:
user_product_part4 = user_product_id[user_product_id['user_id'].isin(user4['user_id']) ]

In [ ]:
x4=apply_apriori_fun(user_product_part4[:int(len(user_product_part4)*0.7)],0.1,0.1)
x4

In [ ]:
apply_apriori_fun(user_product_part4[-int(len(user_product_part4)*0.3):],0.1,0.1)

In [ ]:
# output_rules0 = pd.DataFrame(x0)
# output_rules0.columns=['first','second']
# output_rules0['label']=0


# output_rules1 = pd.DataFrame(x1)
# output_rules1.columns=['first','second']
# output_rules1['label']=1

# output_rules2 = pd.DataFrame(x2)
# output_rules2.columns=['first','second']
# output_rules2['label']=2

# output_rules3 = pd.DataFrame(x3)
# output_rules3.columns=['first','second']
# output_rules3['label']=3

# output_rules4 = pd.DataFrame(x4)
# output_rules4.columns=['first','second']
# output_rules4['label']=4

# output_rules = pd.concat([output_rules0,output_rules1,output_rules2,output_rules3,output_rules4],ignore_index=True)
# output_rules

In [ ]:
# output_rules.to_csv('./output_Aprioir.csv')

In [ ]:
# output=[]
# for item in output_rules.values:
#     arr = []
#     arr.append(list(item[0]))
#     arr.append(list(item[1]))
#     arr.append(item[2])
#     output.append(arr)
# output

In [ ]:
# x = pd.DataFrame(output)
# x.columns=['first','second','label']
# x.to_csv('./output_Aprioir.csv')
# x